In [30]:
#Download BGC Argo Floats data


In [15]:
import polars as pl
import requests
import shutil
import os
import gzip
from pathlib import Path
import re
from tqdm import tqdm

In [4]:
host = 'https://data-argo.ifremer.fr/argo_synthetic-profile_index.txt'
dac = 'https://data-argo.ifremer.fr/dac'
data_directory = 'C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data'
filename = 'synth_file.txt'
destination = os.path.join(data_directory, filename)

In [66]:
response = requests.get(host, stream=True)

with open(destination, "wb") as f:
    r = requests.get(host)
    f.write(r.content)


In [5]:
index_table = pl.read_csv(destination, skip_rows=8)
index_table.height

306621

In [6]:
def extract_digits(text):
    match = re.search(r'\d+', text)
    return match.group() if match else 0

In [7]:
# Mutate a new column based on the regular expression extraction
index_table = index_table.with_columns(
    pl.col('file').map_elements(lambda x: extract_digits(x), return_dtype=pl.Utf8).alias('wmo')
)
print(index_table)

shape: (306_621, 11)
┌────────────┬────────────┬──────────┬───────────┬───┬───────────┬───────────┬───────────┬─────────┐
│ file       ┆ date       ┆ latitude ┆ longitude ┆ … ┆ parameter ┆ parameter ┆ date_upda ┆ wmo     │
│ ---        ┆ ---        ┆ ---      ┆ ---       ┆   ┆ s         ┆ _data_mod ┆ te        ┆ ---     │
│ str        ┆ i64        ┆ f64      ┆ f64       ┆   ┆ ---       ┆ e         ┆ ---       ┆ str     │
│            ┆            ┆          ┆           ┆   ┆ str       ┆ ---       ┆ i64       ┆         │
│            ┆            ┆          ┆           ┆   ┆           ┆ str       ┆           ┆         │
╞════════════╪════════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═════════╡
│ aoml/19007 ┆ 2006102202 ┆ -40.316  ┆ 73.389    ┆ … ┆ PRES TEMP ┆ DDDD      ┆ 202206280 ┆ 1900722 │
│ 22/profile ┆ 1624       ┆          ┆           ┆   ┆ PSAL DOXY ┆           ┆ 80801     ┆         │
│ s/SD190072 ┆            ┆          ┆           ┆   ┆           ┆    

In [26]:
wmo = '4902685'
wmo_table = index_table.filter(pl.col('wmo') == wmo)
wmo_table.height

19

In [27]:
print(wmo_table['file'][1])

meds/4902685/profiles/SR4902685_001D.nc


In [28]:
local_argo_directory = data_directory + '/argo_nc'
wmo_directory = local_argo_directory + '/' + wmo
dac = 'https://data-argo.ifremer.fr/dac'
Path(wmo_directory).mkdir(parents=True, exist_ok=True)



In [29]:
from urllib.request import urlretrieve
for i in tqdm(wmo_table['file']):
    download_url = dac + '/' + i
    filename = wmo_directory + '/' + download_url.rsplit('/', 1)[1]
    urlretrieve(download_url, filename)



100%|██████████| 19/19 [00:13<00:00,  1.44it/s]
